Dependencies

In [ ]:
!pip install numpy

In [ ]:
!pip install --upgrade --quiet azure-cosmos langchain-openai langchain-community


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.1/252.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.3/194.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [ ]:
OPENAI_API_KEY = "key"
OPENAI_API_TYPE = "azure"
OPENAI_API_BASE = "https://test-openai-two-wcb.openai.azure.com/"
OPENAI_EMBEDDINGS_MODEL_NAME = "text-embedding-ada-002"
OPENAI_EMBEDDINGS_MODEL_DEPLOYMENT = "text-embedding-ada-002"

In [ ]:
indexing_policy = {
    "indexingMode": "consistent",
    "includedPaths": [{"path": "/*"}],
    "excludedPaths": [{"path": '/"_etag"/?'}],
    "vectorIndexes": [{"path": "/embedding", "type": "quantizedFlat"}],
}

vector_embedding_policy = {
    "vectorEmbeddings": [
        {
            "path": "/embedding",
            "dataType": "float32",
            "distanceFunction": "cosine",
            "dimensions": 1536,
        }
    ]
}

cosmos_database_properties = {"id": "sem-search"}

In [ ]:
!pip install fpdf
!pip install pypdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40702 sha256=34370804e06de8b0b666175c391c70d9ab08d3fb9fcffdeddc7c5d7c299f8aef
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 5.0 MB/s eta 0:00:00


Formatting JSON Data:

Convert the data from json objects to paragraph format for the model to easily understand the context. The data is also chunked by attribute for deeper understanding.

In [ ]:
import json

# Read the JSON file
with open('spotify.json', 'r') as file:
    json_arr = json.load(file)

def transform_song_data(song):
    attributes = []

    track_name = song.get('track_name')
    attributes.append(f"The name of the track is {track_name}.")

    artists = song.get('artist(s)_name')
    artist_count = song.get('artist_count')
    attributes.append(f"The name of the artist is {artists} ({artist_count}).")

    released_date = f"{song.get('released_year')}-{song.get('released_month')}-{song.get('released_day')}"
    attributes.append(f"This song was released on {released_date}.")

    in_spotify_playlists = song.get('in_spotify_playlists')
    in_spotify_charts = song.get('in_spotify_charts')
    streams = song.get('streams')
    attributes.append(f"Spotify: {in_spotify_playlists} playlists, {in_spotify_charts} charts, {streams} streams.")

    in_apple_playlists = song.get('in_apple_playlists')
    in_apple_charts = song.get('in_apple_charts')
    attributes.append(f"Apple Music: {in_apple_playlists} playlists, {in_apple_charts} charts.")

    in_deezer_playlists = song.get('in_deezer_playlists')
    in_deezer_charts = song.get('in_deezer_charts')
    attributes.append(f"Deezer: {in_deezer_playlists} playlists, {in_deezer_charts} charts.")

    in_shazam_charts = song.get('in_shazam_charts')
    attributes.append(f"Shazam: {in_shazam_charts} charts.")

    bpm = song.get('bpm')
    attributes.append(f"BPM: This song has {bpm} beats per minute.")

    key = song.get('key')
    mode = song.get('mode')
    attributes.append(f"Key: This song is in the key of {key} {mode}.")

    danceability = song.get('danceability_%')
    if danceability >= 80:
        danceability_text = "great for dancing. Music that is often danced to."
    elif 50 <= danceability < 80:
        danceability_text = f"decent for dancing with a danceability of {danceability}."
    else:
        danceability_text = f"terrible for dancing with a danceability of {danceability}."
    attributes.append(f"Danceability: This song is {danceability_text}.")

    valence = song.get('valence_%')
    if valence >= 80:
        valence_text = "very positive vibes"
    elif 50 <= valence < 80:
        valence_text = "moderately positive vibes"
    else:
        valence_text = "low valence vibes"
    attributes.append(f"Valence: This song has {valence_text}.")

    energy = song.get('energy_%')
    if energy >= 80:
        energy_text = "very energetic"
    elif 50 <= energy < 80:
        energy_text = "moderately energetic"
    else:
        energy_text = "low energy"
    attributes.append(f"Energy: This song is {energy_text}.")

    acousticness = song.get('acousticness_%')
    if acousticness >= 50:
        acousticness_text = "highly acoustic"
    else:
        acousticness_text = "not very acoustic"
    attributes.append(f"Acousticness: This song is {acousticness_text}.")

    instrumentalness = song.get('instrumentalness_%')
    if instrumentalness > 0:
        instrumentalness_text = "instrumental"
    else:
        instrumentalness_text = "not instrumental"
    attributes.append(f"Instrumentalness: This song is {instrumentalness_text}.")

    liveness = song.get('liveness_%')
    if liveness >= 50:
        liveness_text = "very lively"
    else:
        liveness_text = "not very lively"
    attributes.append(f"Liveness: This song is {liveness_text}.")

    speechiness = song.get('speechiness_%')
    if speechiness >= 50:
        speechiness_text = "contains a lot of spoken words"
    else:
        speechiness_text = "contains few spoken words"
    attributes.append(f"Speechiness: This song {speechiness_text}.")

    return attributes

def main(json_arr):
    transformed_songs = []
    for song in json_arr:
        transformed_songs.extend(transform_song_data(song))
    return transformed_songs

transformed_songs = main(json_arr)
print(transformed_songs)

['The name of the track is Seven (feat. Latto) (Explicit Ver.).', 'The name of the artist is Latto, Jung Kook (2).', 'This song was released on 2023-7-14.', 'Spotify: 553 playlists, 147 charts, 141381703 streams.', 'Apple Music: 43 playlists, 263 charts.', 'Deezer: 45 playlists, 10 charts.', 'Shazam: 826 charts.', 'BPM: This song has 125 beats per minute.', 'Key: This song is in the key of B Major.', 'Danceability: This song is great for dancing. Music that is often danced to..', 'Valence: This song has very positive vibes.', 'Energy: This song is very energetic.', 'Acousticness: This song is not very acoustic.', 'Instrumentalness: This song is not instrumental.', 'Liveness: This song is not very lively.', 'Speechiness: This song contains few spoken words.', 'The name of the track is LALA.', 'The name of the artist is Myke Towers (1).', 'This song was released on 2023-3-23.', 'Spotify: 1474 playlists, 48 charts, 133716286 streams.', 'Apple Music: 48 playlists, 126 charts.', 'Deezer: 58

Create Documents and Setup Azure Enviornment:

Setup Azure Open AI and Cosmos enviornment by intializing keys and endpoints. Also creates document objects to be passed in embedding model. The documents are created such that each chunk has its song data attached as metadata.

In [ ]:
from azure.cosmos import CosmosClient, PartitionKey
from langchain_community.vectorstores.azure_cosmos_db_no_sql import (
    AzureCosmosDBNoSqlVectorSearch,
)
from langchain_openai import AzureOpenAIEmbeddings
from langchain.docstore.document import Document

cosmos_database_properties = {"id": "sem-search"}
OPENAI_API_VERSION = "2024-02-15-preview"
HOST = "https://chatbot-semantic-db.documents.azure.com:443/"
KEY = "key=="

cosmos_client = CosmosClient(HOST, KEY)
database_name = "sem-search"
container_name = "sem-search-container"
partition_key = PartitionKey(path="/pkey")
cosmos_container_properties = {"partition_key": partition_key}

openai_embeddings = AzureOpenAIEmbeddings(
    azure_deployment=OPENAI_EMBEDDINGS_MODEL_DEPLOYMENT,
    api_version=OPENAI_API_VERSION,
    azure_endpoint=OPENAI_API_BASE,
    openai_api_key=OPENAI_API_KEY,
)

documents = []
index = 0
iterate = 0

for song in transformed_songs:
    document = Document(page_content=song, metadata= json_arr[index // 16])
    index += 1
    documents.append(document)

docs1 = documents[len(documents) // 2:]
docs2 = documents[:len(documents) // 2]



Creates the embeddings and pushed the data to CosmosDB

Don't run this unless data hasn't been embedded and pushed to DB yet.

In [ ]:
from azure.cosmos import CosmosClient, PartitionKey
from langchain_community.vectorstores.azure_cosmos_db_no_sql import (
    AzureCosmosDBNoSqlVectorSearch,)
from langchain_openai import AzureOpenAIEmbeddings
from langchain.docstore.document import Document

vector_search = AzureCosmosDBNoSqlVectorSearch.from_documents(
    documents=docs1,
    embedding=openai_embeddings,
    cosmos_client=cosmos_client,
    database_name=database_name,
    container_name=container_name,
    vector_embedding_policy=vector_embedding_policy,
    indexing_policy=indexing_policy,
    cosmos_container_properties=cosmos_container_properties,
    cosmos_database_properties=cosmos_database_properties,
)

Connects to the Database and the Container

In [ ]:
from azure.cosmos import CosmosClient, exceptions, PartitionKey

COSMOS_DB_URI = 'https://sem-search-db.documents.azure.com:443/'
COSMOS_DB_KEY = 'key=='

client = CosmosClient(COSMOS_DB_URI, COSMOS_DB_KEY)

DATABASE_NAME = 'sem-search'
CONTAINER_NAME = 'sem-search-container'

try:
    database = client.get_database_client(DATABASE_NAME)
    print(f'Database with id {DATABASE_NAME} already exists.')

    container = database.get_container_client(CONTAINER_NAME)
    print(f'Container with id {CONTAINER_NAME} already exists.')
except exceptions.CosmosResourceNotFoundError:
    database = client.create_database(DATABASE_NAME)

Database with id sem-search already exists.
Container with id sem-search-container already exists.


Embeds the query from the user and formats it to be passed to DB

In [ ]:
from openai import OpenAI
import numpy as np

user_query = "Give me the highest streaming songs"

client = OpenAI(api_key = 'sk-proj-key')


response = client.embeddings.create(
  model="text-embedding-ada-002",
  input= user_query,
)

embeddings_array = response.data[0].embedding

embeddings_array = np.array(embeddings_array)

embeddings_array = embeddings_array.tolist()



The following cell tries to use query augmentation to filter results even more. It uses another LLM to generate a WHERE clause for the Database to refine the results based on the original user query

In [ ]:
prompt = f'''
    Create a WHERE clause for the following SQL query that filters the results based on the user's intent and the attributes of the songs stored in a NoSQL Cosmos DB. The attributes include:
    - track_name
    - artist(s)_name
    - artist_count
    - released_year
    - released_month
    - released_day
    - in_spotify_playlists
    - in_spotify_charts
    - streams
    - in_apple_playlists
    - in_apple_charts
    - in_deezer_playlists
    - in_deezer_charts
    - in_shazam_charts
    - bpm
    - key
    - mode
    - danceability_%
    - valence_%
    - energy_%
    - acousticness_%
    - instrumentalness_%
    - liveness_%
    - speechiness_%

    User query: "{user_query}"

    Based on the user's query, generate the appropriate WHERE clause to filter the songs accordingly.

    format the clause as such:
    EXAMPLE: WHERE c.attribute = value
    Don't add anything else

    '''

messages = []
messages.append({"role": "system", "content": prompt})



response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
    )

response = response.choices[0].message.content

response = response.replace("\n", " ")

response

'WHERE c.streams IS NOT NULL'

Connects to the Cosmos DB container and passes a vector similarity query that retrieves the most similar responses to the query vector

In [ ]:
num_items = 5

for item in container.query_items(
            query = f'''
              SELECT TOP {num_items} c.text, c.mode, c.track_name, c.released_year, c["danceability_%"] AS danceability, c["energy_%"] AS energy, c["artist(s)_name"] AS artist_name, c.key, VectorDistance(c.embedding, @embedding) AS SimilarityScore
              FROM c
              ORDER BY VectorDistance(c.embedding, @embedding)
              ''',
            parameters=[
              {"name": "@embedding", "value": embeddings_array}
            ],
            enable_cross_partition_query=True):
  print(json.dumps(item, indent=True))

{
 "text": "Spotify: 3423 playlists, 26 charts, 775542072 streams.",
 "mode": "Minor",
 "track_name": "Lo Siento BB:/ (with Bad Bunny & Julieta Venegas)",
 "released_year": 2021,
 "danceability": 64,
 "energy": 70,
 "artist_name": "Julieta Venegas, Bad Bunny, Tainy",
 "key": "E",
 "SimilarityScore": 0.8089421227304889
}
{
 "text": "Spotify: 6821 playlists, 34 charts, 809306935 streams.",
 "mode": "Minor",
 "track_name": "Desesperados",
 "released_year": 2021,
 "danceability": 87,
 "energy": 69,
 "artist_name": "Chencho Corleone, Rauw Alejandro",
 "key": "C#",
 "SimilarityScore": 0.8072738246003824
}
{
 "text": "Spotify: 15563 playlists, 6 charts, 1887039593 streams.",
 "mode": "Minor",
 "track_name": "good 4 u",
 "released_year": 2021,
 "danceability": 56,
 "energy": 66,
 "artist_name": "Olivia Rodrigo",
 "key": "F#",
 "SimilarityScore": 0.8072058969299291
}
{
 "text": "Spotify: 12329 playlists, 0 charts, 1735441776 streams.",
 "mode": "Minor",
 "track_name": "MONTERO (Call Me By Your 

After Data is migrated to CosmosDB use the code below to query